# Make An Example Digital RF Data Set

This notebook will make a digital RF data set and different parts of the notebook can be subbed out. The other aspect of this is the notebook will save data to complex integers, which will be neccesary when feeding data to a transmitter. Generally codes will abstract out the conversion from one data type to the other.

In [21]:
from pathlib import Path
import numpy as np
import shutil
import time
from fractions import Fraction
import matplotlib.pyplot as plt
import digital_rf as drf

## Make Digital RF Writer Objects

Generally when radios are receiving and writing data we have an object that writes the samples to a file. This function simply encapsulates this so it can be a one line call. Digital RF requires a fair amount of input to things set up. 

In [22]:
def setupwriter(sr_num, sr_den, start_indx, maindir, is_complex, dtype, chaninfo,num_subchannels=1):
    """Sets up the digital_rf writer objects for each channel.

    Parameters
    ----------
    sr_num : int
        Numerator of the sample rate.
    sr_den : int
        Denomenator of the sample rate.
    start_indx : int
        Start time in sample rate * posix time.
    maindir : Path
        Location of the director where the data will be stored.
    is_complex : bool
        Complex data flag.
    dtype : type
        The number type that will be written.
    chaninfo : list
        List of names for the subchanels.
    
    Returns
    -------
    drf_out : DigitalRFWriter
        Object for digital rf to write to file.
    """

    idir = maindir.joinpath("txchan0")
    if idir.is_dir():
        shutil.rmtree(str(idir))
    idir.mkdir(parents=True, exist_ok=True)

    sub_cadence_secs = 3600  # Number of seconds of data in a subdirectory
    file_cadence_millisecs = 1000  # Each file will have up to 400 ms of data
    compression_level = 0  # no compression
    checksum = False  # no checksum

    is_continuous = True
    marching_periods = False  # no marching periods when writing

    uuid = "meptxdata"

    drf_out = drf.DigitalRFWriter(
        str(idir),
        dtype,
        sub_cadence_secs,
        file_cadence_millisecs,
        start_indx,
        sr_num,
        sr_den,
        uuid,
        compression_level,
        checksum,
        is_complex,
        num_subchannels,
        is_continuous,
        marching_periods,
    )

    # Create meta data
    metadir = idir.joinpath("metadata")

    if metadir.is_dir():
        shutil.rmtree(str(metadir))
    metadir.mkdir(parents=True, exist_ok=True)

    adc_obj = drf.DigitalMetadataWriter(
        str(metadir),
        sub_cadence_secs,
        int(file_cadence_millisecs / 1000),
        sr_num,
        sr_den,
        "adc",
    )

        
    adc_obj.write(start_indx, chaninfo)
    return drf_out, adc_obj

In [23]:
def create_data(sr_num,sr_den,start_idx,n_len,c_f, phase0=1+0j,**kwargs):
    """Function to create data

    Parameters
    ----------

    Returns
    -------

    
    """
    data = np.ones(n_len,dtype=np.complex64)
    n_vec = np.arange(n_len,dtype=np.float32)
    sr= Fraction(sr_num,sr_den)
    mod_v = np.exp(2j*np.pi*n_vec*c_f/float(sr))
    
    return data*mod_v*phase0

In [28]:
curfold = Path('.').absolute()

main_dir= curfold.joinpath('example')
dtype = np.dtype([('r', '<i2'), ('i', '<i2')])
sr_num=1000000
sr_den = 1
cf = 100000.
start_indx = drf.util.time_to_sample(time.time(),sr_num)
is_complex=True

chaninfo = {'sample_rate_denominator':sr_den, 'sample_rate_numerator':sr_num,'center_frequencies':np.array([cf,0])}
drfwobj, _ = setupwriter(sr_num, sr_den, start_indx, main_dir, is_complex, dtype, chaninfo)



In [29]:
n_secs = 120
sr = Fraction(sr_num,sr_den)

n_samples = int(n_secs*sr)

n_write = 1000000
mult_data = 32

phase0=1+0j
for st_samp in np.arange(0,n_samples,n_write):
    out_data = create_data(sr_num,sr_den,start_indx+st_samp,n_write+1,cf, phase0)
    phase0 = mult_data*out_data[-1]/np.abs(out_data[-1])
    write_data = np.full(out_data[:-1].shape,0,dtype=dtype)
    write_data[:]['r']= out_data[:-1].real.astype(np.int16)
    write_data[:]['i']= out_data[:-1].imag.astype(np.int16)
    drfwobj.rf_write(write_data)
drfwobj.close()
    

In [30]:
drfObj = drf.DigitalRFReader(str(main_dir))
chans = drfObj.get_channels()
bnds = drfObj.get_bounds(chans[0])
md = drfObj.get_digital_metadata(chans[0])
print(md.read_flatdict())

{'index': 1731088016633162, 'center_frequencies': array([100000.,      0.]), 'sample_rate_denominator': 1, 'sample_rate_numerator': 1000000}
